In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Libreria para estimar regresión
import statsmodels.formula.api as sm

# Libreria para normalizar
from sklearn.preprocessing import StandardScaler

**OBJETIVO**

Predecir la probabilidad de que una persona padezca un problema cardíaco en base a su estado de salud en general, diferenciando si es hombre o mujer 

**CONTEXTO PROFESIONAL**

El área de Cardiología de la Clínica Pergamino se encuentra analizando, a partir de historias médicas de distintos pacientes que fueron atendidos en los consultorios, qué tan probable puede ser que haya muchos pacientes potenciales a sufrir problemas de corazón. 


**PROBLEMÁTICA DEL ÁREA**

Para poder estar lo mejor preparados e invertir en tecnologías que ayuden a intervenir a pacientes con problemas cardiovasculares, es necesario obtener la probabilidad de que existan potenciales personas riesgosas a tener dicha afección. 

**CONTEXTO ANALÍTICO**

Se cuenta con la siguiente información: 
    
* Salud_general
* Último control realizado
* Ejercicio
* Enfermedad del corazón preexistente
* Cáncer de piel
* Otro cáncer
* Depresión
* Diabetes
* Artritis
* Sexo
* Categoría de edad
* Altura (cm)
* Peso (kg)
* IMC
* Historial de tabaquismo
* Consumo de alcohol
* Consumo de frutas
* Consumo de verduras verdes
* Consumo de papas fritas